In [ ]:
# !gdown 1jwJqh3CA6Grcykoh-AA8LlUfyamdFuty

In [1]:
# !pip install sklearn_crfsuite
import joblib
import pandas as pd
model = joblib.load("model.joblib")

In [2]:
stopwords = ["ผู้", "ที่", "ซึ่ง", "อัน"]

def tokens_to_features(tokens, i):
  word = tokens[i]

  features = {
    "bias": 1.0,
    "word.word": word,
    "word[:3]": word[:3],
    "word.isspace()": word.isspace(),
    "word.is_stopword()": word in stopwords,
    "word.isdigit()": word.isdigit(),
    "word.islen5": word.isdigit() and len(word) == 5
  }

  if i > 0:
    prevword = tokens[i - 1]
    features.update({
      "-1.word.prevword": prevword,
      "-1.word.isspace()": prevword.isspace(),
      "-1.word.is_stopword()": prevword in stopwords,
      "-1.word.isdigit()": prevword.isdigit(),
    })
  else:
    features["BOS"] = True

  if i < len(tokens) - 1:
    nextword = tokens[i + 1]
    features.update({
      "+1.word.nextword": nextword,
      "+1.word.isspace()": nextword.isspace(),
      "+1.word.is_stopword()": nextword in stopwords,
      "+1.word.isdigit()": nextword.isdigit(),
    })
  else:
    features["EOS"] = True

  return features

def parse(text):
  tokens = text.split()
  features = [tokens_to_features(tokens, i) for i in range(len(tokens))]
  print(model.predict([features])[0])

In [3]:
text = "นายมงคล 123/4 กรุงเทพมหานคร ซึ่งมีพื้นที่ 100 ไร่"
tokens = text.split()
features = [tokens_to_features(tokens, i) for i in range(len(tokens))]

In [4]:
features

[{'bias': 1.0,
  'word.word': 'นายมงคล',
  'word[:3]': 'นาย',
  'word.isspace()': False,
  'word.is_stopword()': False,
  'word.isdigit()': False,
  'word.islen5': False,
  'BOS': True,
  '+1.word.nextword': '123/4',
  '+1.word.isspace()': False,
  '+1.word.is_stopword()': False,
  '+1.word.isdigit()': False},
 {'bias': 1.0,
  'word.word': '123/4',
  'word[:3]': '123',
  'word.isspace()': False,
  'word.is_stopword()': False,
  'word.isdigit()': False,
  'word.islen5': False,
  '-1.word.prevword': 'นายมงคล',
  '-1.word.isspace()': False,
  '-1.word.is_stopword()': False,
  '-1.word.isdigit()': False,
  '+1.word.nextword': 'กรุงเทพมหานคร',
  '+1.word.isspace()': False,
  '+1.word.is_stopword()': False,
  '+1.word.isdigit()': False},
 {'bias': 1.0,
  'word.word': 'กรุงเทพมหานคร',
  'word[:3]': 'กรุ',
  'word.isspace()': False,
  'word.is_stopword()': False,
  'word.isdigit()': False,
  'word.islen5': False,
  '-1.word.prevword': '123/4',
  '-1.word.isspace()': False,
  '-1.word.is_stopwo

In [8]:
model.predict([features])

array([['O', 'O', 'LOC', 'LOC', 'ADDR', 'ADDR']], dtype=object)

In [9]:
model

CRF(algorithm='lbfgs', all_possible_transitions=True,
    c1=np.float64(0.1540557661329477), c2=np.float64(0.022049164379631885),
    max_iterations=100)

In [10]:
from pprint import pprint
text = 'นายสมชาย เข็มกลัด 254 ถนน พญาไท แขวง วังใหม่ เขต ปทุมวัน กรุงเทพมหานคร 10330'
tokens = text.split()
for i in range(len(tokens)):
  pprint(tokens_to_features(text, i))
  print('')

{'+1.word.is_stopword()': False,
 '+1.word.isdigit()': False,
 '+1.word.isspace()': False,
 '+1.word.nextword': 'า',
 'BOS': True,
 'bias': 1.0,
 'word.is_stopword()': False,
 'word.isdigit()': False,
 'word.islen5': False,
 'word.isspace()': False,
 'word.word': 'น',
 'word[:3]': 'น'}

{'+1.word.is_stopword()': False,
 '+1.word.isdigit()': False,
 '+1.word.isspace()': False,
 '+1.word.nextword': 'ย',
 '-1.word.is_stopword()': False,
 '-1.word.isdigit()': False,
 '-1.word.isspace()': False,
 '-1.word.prevword': 'น',
 'bias': 1.0,
 'word.is_stopword()': False,
 'word.isdigit()': False,
 'word.islen5': False,
 'word.isspace()': False,
 'word.word': 'า',
 'word[:3]': 'า'}

{'+1.word.is_stopword()': False,
 '+1.word.isdigit()': False,
 '+1.word.isspace()': False,
 '+1.word.nextword': 'ส',
 '-1.word.is_stopword()': False,
 '-1.word.isdigit()': False,
 '-1.word.isspace()': False,
 '-1.word.prevword': 'า',
 'bias': 1.0,
 'word.is_stopword()': False,
 'word.isdigit()': False,
 'word.islen5': F

In [11]:
parse("นายสมชาย เข็มกลัด 254 ถนน พญาไท แขวง วังใหม่ เขต ปทุมวัน กรุงเทพมหานคร 10330")

['O' 'O' 'ADDR' 'ADDR' 'ADDR' 'ADDR' 'ADDR' 'ADDR' 'LOC' 'LOC' 'POST']


In [13]:
parse("นายมงคล 123/4 ตำบล บ้านไกล อำเภอ เมือง จังหวัด ลพบุรี 15000")

['ADDR' 'ADDR' 'ADDR' 'ADDR' 'ADDR' 'ADDR' 'ADDR' 'ADDR' 'POST']


In [14]:
parse("นายมงคล 123/4 กรุงเทพมหานคร ซึ่งมีพื้นที่ 100 ไร่")

['O' 'O' 'LOC' 'LOC' 'ADDR' 'ADDR']


In [20]:
import requests
import json
import pandas as pd